
# 📘 XGBoost Classification
---

# 1️⃣ Goal
Understand how **XGBoost solves classification problems step-by-step**.

Toy Problem:
Predict **placement (0/1)** using **CGPA**.

---

# 2️⃣ Prerequisites
You should know:
- Gradient Boosting
- Logistic Regression basics
- Log-Odds concept

---

# 3️⃣ Big Idea

XGBoost classification workflow = Gradient Boosting workflow

### Core Loop
1. Start with base prediction (log-odds)
2. Compute residuals
3. Train tree on residuals
4. Add scaled tree output
5. Repeat

Key difference:
> Trees are built using **similarity score**, not entropy/Gini

---

# 4️⃣ Stage 1 — Base Model

In regression → mean  
In classification → **log-odds**

### Log-Odds Formula
```
log(p / (1 - p))
```

Where:
p = probability of positive class

---

# 5️⃣ Compute Base Log-Odds

Assume:
- 3 positives
- 2 negatives

```
p = 3/5
log-odds = log(3/2) ≈ 0.405
```

So base prediction for all samples:
```
F₀(x) = 0.405
```

---

# 6️⃣ Convert Log-Odds → Probability

Using sigmoid:

```
p = e^F / (1 + e^F)
```

For 0.405:
```
p ≈ 0.60
```

So model predicts:
> 60% placement probability for everyone

Clearly weak model ❌

---

# 7️⃣ Residuals (Pseudo-Residuals)

Residual formula:
```
Residual = Actual − Predicted Probability
```

Example residuals:
- 1 − 0.6 = +0.4
- 0 − 0.6 = −0.6

Residuals capture:
> Model error direction

---

# 8️⃣ Build First XGBoost Tree

Train tree on:
```
Input: CGPA
Target: Residuals
```

But tree construction is DIFFERENT.

---

# 9️⃣ Similarity Score (Classification Version)

### Formula
```
Similarity = Σ(residual²) / Σ(p * (1 - p)) + λ
```

Where:
- p = previous probability
- λ = regularization (assume 0)

This replaces:
- Gini
- Entropy

---

# 🔟 Root Node Similarity

If residuals cancel out → score ≈ 0

This becomes baseline.

Goal now:
> Split data to increase similarity score

---

# 1️⃣1️⃣ Find Split Points

Sort feature (CGPA).

Candidate splits = averages between adjacent values.

Example:
```
5.97, 6.67, 7.62, 8.87
```

We test ALL splits.

---

# 1️⃣2️⃣ Evaluate Each Split

For each split:

1. Divide residuals
2. Compute similarity for left and right
3. Compute gain

---

# Gain Formula
```
Gain = LeftScore + RightScore − ParentScore
```

Choose split with **max gain**.

---

# 1️⃣3️⃣ Best Split Found

Example best split:
```
CGPA < 7.62
```

Tree structure:
```
        CGPA < 7.62
        /            Residual A   Residual B
```

---

# 1️⃣4️⃣ Leaf Output Formula

Leaf output determines tree contribution.

### Formula
```
Leaf Output = Σ(residuals) / Σ(p * (1 - p)) + λ
```

Difference from similarity score:
- No square in numerator

---

# 1️⃣5️⃣ Example Leaf Outputs

Left leaf:
```
≈ -1.11
```

Right leaf:
```
≈ +1.66
```

These are in **log-odds space**.

---

# 1️⃣6️⃣ Stage 2 Model

Combined model:
```
F(x) = Base Log-Odds + η × Tree Output
```

Where:
η = learning rate (e.g., 0.3)

---

# 1️⃣7️⃣ Example Prediction

If CGPA = 6:
- Goes left leaf
- Output = -1.11

New log-odds:
```
0.405 + 0.3 × (-1.11) = 0.072
```

---

# 1️⃣8️⃣ Convert Back to Probability

Apply sigmoid again:

```
p = e^0.072 / (1 + e^0.072) ≈ 0.518
```

So probability dropped from:
```
0.60 → 0.52
```

Model improved ✅

---

# 1️⃣9️⃣ Compute New Residuals

```
New Residual = Actual − New Probability
```

Residuals shrink toward 0.

This indicates:
> Model learning correctly

---

# 2️⃣0️⃣ Stage 3+

Repeat process:

1. Train new tree on new residuals
2. Add to model
3. Convert to probability
4. Compute new residuals

---

# Final Model Form

```
F(x) = Base + ηT₁ + ηT₂ + ηT₃ ...
```

Prediction:
```
Probability = sigmoid(F(x))
```

---

# 2️⃣1️⃣ Key Differences vs Gradient Boosting

| Aspect | Gradient Boosting | XGBoost |
|-------|------------------|---------|
| Tree split metric | Entropy/Gini | Similarity score |
| Regularization | Weak | Strong |
| Speed | Moderate | Optimized |
| Math | Simpler | Second-order |

---

# 2️⃣2️⃣ Important Formulas

### Log-Odds
```
log(p / (1 - p))
```

### Sigmoid
```
e^F / (1 + e^F)
```

### Similarity Score
```
Σ(residual²) / Σ(p(1 - p))
```

### Gain
```
Left + Right − Parent
```

### Leaf Output
```
Σ(residual) / Σ(p(1 - p))
```

---

# 2️⃣3️⃣ Intuition

- Base model predicts global probability
- Trees fix mistakes
- Each tree nudges predictions
- Residuals shrink toward 0

When residual ≈ 0:
> Model is optimal

---

# 2️⃣4️⃣ Why Log-Odds?

Because:
- Works well with sigmoid
- Stable gradients
- Smooth optimization

This is same idea used in:
- Logistic regression
- Gradient boosting classifier

---

# 2️⃣5️⃣ What Makes XGBoost Powerful

- Log-odds optimization
- Regularized tree building
- Similarity-based splits
- Learning rate shrinkage
- Additive modeling

---
